# PyGMTSAR SBAS and PSI Analyses: Borowa Gora - Poland, 2023

## 1. Load and Setup Python Modules

In [40]:
import platform, sys, os
PATH = os.environ['PATH']
from pygmtsar import __version__
print(__version__)

import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
import json
from dask.distributed import Client
import dask
import warnings
warnings.filterwarnings('ignore')

# plotting modules
import pyvista as pv

# magic trick for white background
pv.set_plot_theme("document")
import panel
panel.extension(comms='ipywidgets')
panel.extension('vtk')
from contextlib import contextmanager
import matplotlib.pyplot as plt
@contextmanager
def mpl_settings(settings):
    original_settings = {k: plt.rcParams[k] for k in settings}
    plt.rcParams.update(settings)
    yield
    plt.rcParams.update(original_settings)
plt.rcParams['figure.figsize'] = [12, 4]
plt.rcParams['figure.dpi'] = 150
plt.rcParams['figure.titlesize'] = 24
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

#%matplotlib inline

# define Pandas display settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 100)

from pygmtsar import S1, Stack, tqdm_dask, ASF, Tiles, XYZTiles, utils

2024.8.30.post5


In [41]:
from os.path import abspath, dirname, join
PROJ_PATH = dirname(abspath(abspath("")))
sys.path.insert(0, PROJ_PATH)

from settings.paths import setup
setup()

## 2. Define Sentinel-1 SLC Scenes and Processing Parameters

### Descending Orbit Configuration

https://search.asf.alaska.edu/#/?polygon=POINT(72.66%2038.25)&start=2017-05-11T00:00:01Z&end=2017-12-25T23:59:59Z&productTypes=SLC&resultsLoaded=true&zoom=7.131&center=74.457,35.638&path=5-5&frame=466-466

In [42]:
# The subswath is required for partial scene downloads and is not used for burst downloads.
# The orbit is used to define directory names.
ORBIT    = 'D'
SUBSWATH = 3
REFERENCE = '2023-06-07'

https://search.asf.alaska.edu/#/?polygon=POLYGON((72.72%2038.26,72.6936%2038.3236,72.63%2038.35,72.5664%2038.3236,72.54%2038.26,72.5664%2038.1964,72.63%2038.17,72.6936%2038.1964,72.72%2038.26))&start=2017-05-11T00:00:01Z&end=2017-12-25T23:59:59Z&resultsLoaded=true&zoom=10.025&center=72.469,38.068&path=5-5&frame=466-466&granule=S1_009440_IW2_20171225T011414_VV_4328-BURST&dataset=SENTINEL-1%20BURSTS&polarizations=VV

In [43]:
BURSTS = """
S1_327247_IW3_20231228T044448_VV_33C2-BURST
S1_327247_IW3_20231216T044449_VV_1CC1-BURST
S1_327247_IW3_20231204T044450_VV_42BA-BURST
S1_327247_IW3_20231122T044450_VV_05AE-BURST
S1_327247_IW3_20231110T044450_VV_B748-BURST
S1_327247_IW3_20231017T044451_VV_49C0-BURST
S1_327247_IW3_20231005T044451_VV_1C05-BURST
S1_327247_IW3_20230923T044451_VV_3F01-BURST
S1_327247_IW3_20230911T044450_VV_E969-BURST
S1_327247_IW3_20230830T044450_VV_90C0-BURST
S1_327247_IW3_20230818T044449_VV_C701-BURST
S1_327247_IW3_20230806T044449_VV_FD68-BURST
S1_327247_IW3_20230725T044448_VV_6BD6-BURST
S1_327247_IW3_20230713T044447_VV_BDC7-BURST
S1_327247_IW3_20230701T044446_VV_FCB0-BURST
S1_327247_IW3_20230619T044446_VV_1703-BURST
S1_327247_IW3_20230607T044445_VV_2A6C-BURST
S1_327247_IW3_20230514T044444_VV_43BE-BURST
S1_327247_IW3_20230526T044444_VV_8A01-BURST
S1_327247_IW3_20230502T044443_VV_C50D-BURST
S1_327247_IW3_20230420T044442_VV_8F60-BURST
S1_327247_IW3_20230408T044442_VV_C9C0-BURST
S1_327247_IW3_20230327T044442_VV_AAD6-BURST
S1_327247_IW3_20230315T044441_VV_147B-BURST
S1_327247_IW3_20230303T044442_VV_DCED-BURST
S1_327247_IW3_20230219T044442_VV_527E-BURST
S1_327247_IW3_20230207T044442_VV_FA04-BURST
S1_327247_IW3_20230126T044442_VV_C08C-BURST
S1_327247_IW3_20230114T044443_VV_1187-BURST
S1_327247_IW3_20230102T044443_VV_D42C-BURST
"""
BURSTS = list(filter(None, BURSTS.split('\n')))
print (f'Bursts defined: {len(BURSTS)}')

Bursts defined: 30


In [12]:
POLAR = 'VV'

In [44]:
from settings.paths import DATA_DIR
main_folder = join(DATA_DIR, 'sar/sbas/desc/2023/bogo_pl')

In [45]:
WORKDIR = os.path.join(main_folder, 'raw')
DATADIR = os.path.join(main_folder, 'data')

In [46]:
# define DEM filename inside data directory
DEM = f'{DATADIR}/dem.nc'

In [52]:
import rasterio

In [53]:
tif_path = '/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_IW_SLC__1SDV_20231228T044448_20231228T044451_051850_06437C_33C2.SAFE/measurement/s1a-iw3-slc-vv-20231228t044448-20231228t044451-051850-06437c-001.tiff'

In [54]:
dat = rasterio.open(tif_path)

In [47]:
# subsidence point from
geojson = '''
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [21.04, 52.475]
  },
  "properties": {}
}
'''
POI = gpd.GeoDataFrame.from_features([json.loads(geojson)])
POI

,geometry
0,POINT (21.04 52.475)


In [48]:
geojson = '''
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [21.04, 52.475]
  },
  "properties": {}
}
'''
BUFFER = 0.09
AOI = gpd.GeoDataFrame.from_features([json.loads(geojson)])
AOI['geometry'] = AOI.buffer(BUFFER)
AOI

,geometry
0,"POLYGON ((21.13 52.475, 21.12957 52.46618, 21.12827 52.45744, 21.12612 52.44887, 21.12315 52.440..."


## Download and Unpack Datasets

## Enter Your ASF User and Password

If the data directory is empty or doesn't exist, you'll need to download Sentinel-1 scenes from the Alaska Satellite Facility (ASF) datastore. Use your Earthdata Login credentials. If you don't have an Earthdata Login, you can create one at https://urs.earthdata.nasa.gov//users/new

You can also use pre-existing SLC scenes stored on your Google Drive, or you can copy them using a direct public link from iCloud Drive.

The credentials below are available at the time the notebook is validated.

In [49]:
from utils.internal.io.json_io import open_json
from settings.paths import KEYS_DIR

secrets = open_json(join(KEYS_DIR, 'secrets.json'))

# Set these variables to None and you will be prompted to enter your username and password below.
asf = ASF(secrets['asf']['username'], secrets['asf']['password'])

In [13]:

# Optimized scene downloading from ASF - only the required subswaths and polarizations.
# Subswaths are already encoded in burst identifiers and are only needed for scenes.
#print(asf.download(DATADIR, SCENES, SUBSWATH))

#for burst in BURSTS:
#    print(asf.download(DATADIR, [burst], skip_exist=True, polarization=POLAR, n_jobs=1))

In [50]:
# scan the data directory for SLC scenes and download missed orbits
S1.download_orbits(DATADIR, S1.scan_slc(DATADIR))

In [51]:
# download Copernicus Global DEM 1 arc-second
Tiles().download_dem(AOI, filename=DEM)

Tiles Parallel Downloading:   0%|          | 0/2 [00:00<?, ?it/s]

## Run Local Dask Cluster

Launch Dask cluster for local and distributed multicore computing. That's possible to process terabyte scale Sentinel-1 SLC datasets on Apple Air 16 GB RAM.

In [14]:
# simple Dask initialization
if 'client' in globals():
    client.close()
client = Client()
client

2025-04-29 06:00:12,093 - distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/home/rav_marcin/anaconda3/envs/msg2sar_env/lib/python3.12/site-packages/distributed/protocol/core.py", line 175, in loads
    return msgpack.loads(
           ^^^^^^^^^^^^^^
  File "/home/rav_marcin/anaconda3/envs/msg2sar_env/lib/python3.12/site-packages/msgpack/fallback.py", line 118, in unpackb
    unpacker = Unpacker(None, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: Unpacker.__init__() got an unexpected keyword argument 'strict_map_key'
2025-04-29 06:00:12,097 - distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/home/rav_marcin/anaconda3/envs/msg2sar_env/lib/python3.12/site-packages/distributed/protocol/core.py", line 175, in loads
    return msgpack.loads(
           ^^^^^^^^^^^^^^
  File "/home/rav_marcin/anaconda3/envs/msg2sar_env/lib/python3.12/site-packages/msgpack/fallback.py", 

TypeError: Unpacker.__init__() got an unexpected keyword argument 'strict_map_key'

## Init

Search recursively for measurement (.tiff) and annotation (.xml) and orbit (.EOF) files in the DATA directory. It can be directory with full unzipped scenes (.SAFE) subdirectories or just a directory with the list of pairs of required .tiff and .xml files (maybe pre-filtered for orbit, polarization and subswath to save disk space). If orbit files and DEM are missed these will be downloaded automatically below.

### Select Original Secenes and Orbits

Use filters to find required subswath, polarization and orbit in original scenes .SAFE directories in the data directory.

In [58]:
scenes = S1.scan_slc(DATADIR)

In [59]:
sbas = Stack(WORKDIR, drop_if_exists=True).set_scenes(scenes).set_reference(REFERENCE)
sbas.to_dataframe()

NOTE: auto set reference scene 2023-01-02. You can change it like Stack.set_reference("2022-01-20")


,datetime,orbit,mission,polarization,subswath,datapath,metapath,noisepath,calibpath,orbitpath,geometry
date,,,,,,,,,,,
2023-01-02,2023-01-02 04:44:43,D,S1A,VV,3,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_IW_SLC__1SDV_20230102...,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_IW_SLC__1SDV_20230102...,None,None,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_OPER_AUX_POEORB_OPOD_...,"MULTIPOLYGON (((21.89212 52.54494, 21.82631 52.55166, 21.7607 52.55832, 21.6955 52.5649, 21.6306..."
2023-01-14,2023-01-14 04:44:43,D,S1A,VV,3,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_IW_SLC__1SDV_20230114...,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_IW_SLC__1SDV_20230114...,None,None,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_OPER_AUX_POEORB_OPOD_...,"MULTIPOLYGON (((21.89429 52.54504, 21.82848 52.55175, 21.76287 52.55841, 21.69767 52.56499, 21.6..."
2023-01-26,2023-01-26 04:44:42,D,S1A,VV,3,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_IW_SLC__1SDV_20230126...,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_IW_SLC__1SDV_20230126...,None,None,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_OPER_AUX_POEORB_OPOD_...,"MULTIPOLYGON (((21.89161 52.54495, 21.8258 52.55167, 21.76018 52.55833, 21.69498 52.56491, 21.63..."
2023-02-07,2023-02-07 04:44:42,D,S1A,VV,3,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_IW_SLC__1SDV_20230207...,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_IW_SLC__1SDV_20230207...,None,None,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_OPER_AUX_POEORB_OPOD_...,"MULTIPOLYGON (((21.89083 52.5454, 21.82502 52.55211, 21.75941 52.55877, 21.69422 52.56535, 21.62..."
2023-02-19,2023-02-19 04:44:42,D,S1A,VV,3,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_IW_SLC__1SDV_20230219...,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_IW_SLC__1SDV_20230219...,None,None,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_OPER_AUX_POEORB_OPOD_...,"MULTIPOLYGON (((21.89117 52.54496, 21.82536 52.55168, 21.75975 52.55833, 21.69455 52.56491, 21.6..."
2023-03-03,2023-03-03 04:44:42,D,S1A,VV,3,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_IW_SLC__1SDV_20230303...,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_IW_SLC__1SDV_20230303...,None,None,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_OPER_AUX_POEORB_OPOD_...,"MULTIPOLYGON (((21.89023 52.54473, 21.82443 52.55144, 21.75882 52.5581, 21.69362 52.56468, 21.62..."
2023-03-15,2023-03-15 04:44:41,D,S1A,VV,3,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_IW_SLC__1SDV_20230315...,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_IW_SLC__1SDV_20230315...,None,None,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_OPER_AUX_POEORB_OPOD_...,"MULTIPOLYGON (((21.89361 52.54508, 21.82781 52.55179, 21.76221 52.55845, 21.69702 52.56503, 21.6..."
2023-03-27,2023-03-27 04:44:42,D,S1A,VV,3,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_IW_SLC__1SDV_20230327...,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_IW_SLC__1SDV_20230327...,None,None,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_OPER_AUX_POEORB_OPOD_...,"MULTIPOLYGON (((21.89102 52.54512, 21.82522 52.55183, 21.75961 52.55849, 21.69442 52.56507, 21.6..."
2023-04-08,2023-04-08 04:44:42,D,S1A,VV,3,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_IW_SLC__1SDV_20230408...,/home/rav_marcin/projects/msg2sar/data/sar/sbas/desc/2023/bogo_pl/data/S1A_IW_SLC__1SDV_20230408...,None,No

In [64]:
geom = sbas.to_dataframe()['geometry']
intersect_p1 = geom.intersection(geom)
geom_idx = geom.index.values.tolist()
poly = intersect_p1[geom_idx[0]]
for i in range(1, len(geom.index)):
    poly = intersect_p1[geom_idx[i]].intersection(poly)

In [66]:
type(poly)

shapely.geometry.polygon.Polygon

In [ ]:
poly.exterior.coords

In [ ]:
sbas.plot_scenes(AOI=AOI)

## Reframe Scenes (Optional)

Stitch sequential scenes and crop the subswath to a smaller area for faster processing when the full area is not needed.

In [ ]:
sbas.compute_reframe(AOI)

In [ ]:
sbas.plot_scenes(AOI=AOI)

### Load DEM

The function below loads DEM from file or Xarray variable and converts heights to ellipsoidal model using EGM96 grid.

In [ ]:
# define the area of interest (AOI) to speedup the processing
sbas.load_dem(DEM, AOI)

In [ ]:
sbas.plot_scenes(AOI=AOI)

## Align Images

In [ ]:
sbas.compute_align()

## Geocoding Transform

In [ ]:
# use the original Sentinel-1 resolution (1 pixel spacing)
sbas.compute_geocode(1)

In [ ]:
sbas.plot_topo(quantile=[0.01, 0.99])

## Persistent Scatterers Function (PSF)

In [ ]:
# use the only selected dates for the pixels stability analysis
sbas.compute_ps()

In [ ]:
sbas.plot_psfunction(quantile=[0.01, 0.90])

In [ ]:
psmask_sbas = sbas.multilooking(sbas.psfunction(), coarsen=(1,4), wavelength=100)>0.5
topo_sbas = sbas.get_topo().interp_like(psmask_sbas, method='nearest')
landmask_sbas = psmask_sbas&(np.isfinite(topo_sbas))
landmask_sbas = utils.binary_opening(landmask_sbas, structure=np.ones((20,20)))
landmask_sbas = np.isfinite(sbas.conncomp_main(landmask_sbas))
landmask_sbas = utils.binary_closing(landmask_sbas, structure=np.ones((20,20)))
landmask_sbas = np.isfinite(psmask_sbas.where(landmask_sbas))
sbas.plot_landmask(landmask_sbas)

## SBAS Baseline

In [ ]:
baseline_pairs = sbas.sbas_pairs(days=60)
# optionally, drop dates having less then 2 pairs
#baseline_pairs = sbas.sbas_pairs_limit(baseline_pairs, limit=2, iterations=2)
# optionally, drop all pairs connected to the specified dates
#baseline_pairs = sbas.sbas_pairs_filter_dates(baseline_pairs, ['2021-01-01'])
baseline_pairs

In [ ]:
with mpl_settings({'figure.dpi': 300}):
    sbas.plot_baseline(baseline_pairs)

## SBAS Analysis

### Multi-looked Resolution for SBAS

In [ ]:
sbas.compute_interferogram_multilook(baseline_pairs, 'intf_mlook', wavelength=200, psize=32,
                                     weight=sbas.psfunction())

In [ ]:
# optionally, materialize to disk and open
ds_sbas = sbas.open_stack('intf_mlook')
# apply land mask
ds_sbas = ds_sbas.where(landmask_sbas)
intf_sbas = ds_sbas.phase
corr_sbas = ds_sbas.correlation
corr_sbas

In [ ]:
sbas.plot_interferograms(intf_sbas[:8], caption='SBAS Phase, [rad]')

In [ ]:
sbas.plot_correlations(corr_sbas[:8], caption='SBAS Correlation')

### Quality Check

In [ ]:
#baseline_pairs['corr'] = corr_sbas.sel(pair=baseline_pairs.pair.values).mean(['y', 'x'])
baseline_pairs['corr'] = corr_sbas.mean(['y', 'x'])
print (len(baseline_pairs))
baseline_pairs

In [ ]:
pairs_best = sbas.sbas_pairs_covering_correlation(baseline_pairs, 2)
print (len(pairs_best))
pairs_best

In [ ]:
with mpl_settings({'figure.dpi': 300}):
    sbas.plot_baseline(pairs_best)

In [ ]:
sbas.plot_baseline_correlation(baseline_pairs, pairs_best)

In [ ]:
sbas.plot_baseline_duration(baseline_pairs, column='corr', ascending=False)

In [ ]:
sbas.plot_baseline_duration(pairs_best, column='corr', ascending=False)

In [ ]:
intf_sbas = intf_sbas.sel(pair=pairs_best.pair.values)
corr_sbas = corr_sbas.sel(pair=pairs_best.pair.values)

In [ ]:
sbas.plot_interferograms(intf_sbas[:8], caption='SBAS Phase, [rad]')

In [ ]:
sbas.plot_correlations(corr_sbas[:8], caption='SBAS Correlation')

### 2D Unwrapping

In [ ]:
corr_sbas_stack = corr_sbas.mean('pair')

In [ ]:
corr_sbas_stack = sbas.sync_cube(corr_sbas_stack, 'corr_sbas_stack')

In [ ]:
sbas.plot_correlation_stack(corr_sbas_stack, CORRLIMIT := 0.3, caption='SBAS Stack Correlation')

In [ ]:
sbas.plot_interferograms(intf_sbas[:8].where(corr_sbas_stack>CORRLIMIT), caption='SBAS Phase, [rad]')

In [ ]:
unwrap_sbas = sbas.unwrap_snaphu(
    intf_sbas.where(corr_sbas_stack>CORRLIMIT),
    corr_sbas,
    conncomp=True
)
unwrap_sbas

In [ ]:
# optionally, materialize to disk and open
unwrap_sbas = sbas.sync_cube(unwrap_sbas, 'unwrap_sbas')

In [ ]:
sbas.plot_phases((unwrap_sbas.phase - unwrap_sbas.phase.mean(['y','x']))[:8], caption='SBAS Phase, [rad]')

In [ ]:
# select the main valid component
unwrap_sbas = sbas.conncomp_main(unwrap_sbas, 1)

In [ ]:
sbas.plot_phases((unwrap_sbas.phase - unwrap_sbas.phase.mean(['y','x']))[:8], caption='SBAS Phase, [rad]')

### Trend Correction

In [ ]:
decimator_sbas = sbas.decimator(resolution=15, grid=(1,1))
topo = decimator_sbas(sbas.get_topo())
yy, xx = xr.broadcast(topo.y, topo.x)
trend_sbas = sbas.regression(unwrap_sbas.phase,
        [topo,    topo*yy,    topo*xx,    topo*yy*xx,
         topo**2, topo**2*yy, topo**2*xx, topo**2*yy*xx,
         yy, xx, yy*xx], corr_sbas)

In [ ]:
# optionally, materialize to disk and open
trend_sbas = sbas.sync_cube(trend_sbas, 'trend_sbas')

In [ ]:
sbas.plot_phases(trend_sbas[:8], caption='SBAS Trend Phase, [rad]', quantile=[0.01, 0.99])

In [ ]:
sbas.plot_phases((unwrap_sbas.phase - trend_sbas)[:8], caption='SBAS Phase - Trend, [rad]', vmin=-np.pi, vmax=np.pi)

### Coherence-Weighted Least-Squares Solution for LOS Displacement, mm

In [ ]:
# calculate phase displacement in radians and convert to LOS displacement in millimeter
disp_sbas = sbas.los_displacement_mm(sbas.lstsq(unwrap_sbas.phase - trend_sbas, corr_sbas))

In [ ]:
# optionally, materialize to disk and open
disp_sbas = sbas.sync_cube(disp_sbas, 'disp_sbas')

In [ ]:
sbas.plot_displacements(disp_sbas[::3], caption='SBAS Cumulative LOS Displacement, [mm]',
                        quantile=[0.01, 0.99], symmetrical=True)

### Least-squares model for LOS Displacement, mm

In [ ]:
velocity_sbas = sbas.velocity(disp_sbas)
velocity_sbas

In [ ]:
# optionally, materialize to disk and open
velocity_sbas = sbas.sync_cube(velocity_sbas, 'velocity_sbas')

In [ ]:
fig = plt.figure(figsize=(12,4), dpi=300)

zmin, zmax = np.nanquantile(velocity_sbas, [0.01, 0.99])
zminmax = max(abs(zmin), zmax)

ax = fig.add_subplot(1, 2, 1)
velocity_sbas.plot.imshow(cmap='turbo', vmin=-zminmax, vmax=zminmax, ax=ax)
sbas.geocode(AOI.boundary).plot(ax=ax)
sbas.geocode(POI).plot(ax=ax, marker='x', c='r', markersize=100, label='POI')
ax.set_aspect('auto')
ax.set_title('Velocity, mm/year', fontsize=16)

ax = fig.add_subplot(1, 2, 2)
sbas.as_geo(sbas.ra2ll(velocity_sbas)).rio.clip(AOI.geometry)\
    .plot.imshow(cmap='turbo', vmin=-zminmax, vmax=zminmax, ax=ax)
AOI.boundary.plot(ax=ax)
POI.plot(ax=ax, marker='x', c='r', markersize=100, label='POI')
ax.legend(loc='upper left', fontsize=14)
ax.set_title('Velocity, mm/year', fontsize=16)

plt.suptitle('SBAS LOS Velocity, 2021', fontsize=18)
plt.tight_layout()
plt.show()

### STL model for LOS Displacement, mm

In [ ]:
plt.figure(figsize=(12, 4), dpi=300)

x, y = [(geom.x, geom.y) for geom in sbas.geocode(POI).geometry][0]
disp_pixel = disp_sbas.sel(y=y, x=x, method='nearest')
stl_pixel = sbas.stl(disp_sbas.sel(y=[y], x=[x], method='nearest')).isel(x=0, y=0)
plt.plot(disp_pixel.date, disp_pixel, c='r', lw=2, label='Displacement POI')
plt.plot(stl_pixel.date, stl_pixel.trend, c='r', ls='--', lw=2, label='Trend POI')
plt.plot(stl_pixel.date, stl_pixel.seasonal, c='r', lw=1, label='Seasonal POI')

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=14)
plt.title('SBAS LOS Displacement STL Decompose, 2021', fontsize=18)
plt.ylabel('Displacement, mm', fontsize=16)
plt.show()

## PS Analysis

Use the trend detected on possibly lower resolution unwrapped phases for higher resolution analysis.

In [ ]:
stability = sbas.psfunction()
landmask_ps = landmask_sbas.astype(int).interp_like(stability, method='nearest').astype(bool)
sbas.compute_interferogram_singlelook(pairs_best, 'intf_slook', wavelength=60,
                                      weight=stability.where(landmask_ps), phase=trend_sbas)

In [ ]:
# optionally, materialize to disk and open
ds_ps = sbas.open_stack('intf_slook')
intf_ps = ds_ps.phase
corr_ps = ds_ps.correlation

In [ ]:
sbas.plot_interferograms(intf_ps[:8], caption='PS Phase, [rad]')

In [ ]:
sbas.plot_correlations(corr_ps[:8], caption='PS Correlation')

### 1D Unwrapping and LOS Displacement, mm

In [ ]:
disp_ps_pairs = sbas.los_displacement_mm(sbas.unwrap1d(intf_ps))
disp_ps_pairs

In [ ]:
# optionally, materialize to disk and open
disp_ps_pairs = sbas.sync_cube(disp_ps_pairs, 'disp_ps_pairs')

### Coherence-Weighted Least-Squares Solution for LOS Displacement, mm

In [ ]:
disp_ps = sbas.lstsq(disp_ps_pairs, corr_ps)
disp_ps

In [ ]:
# optionally, materialize to disk and open
disp_ps = sbas.sync_cube(disp_ps, 'disp_ps')

In [ ]:
sbas.plot_displacements(disp_ps[::3], caption='PS Cumulative LOS Displacement, [mm]',
                        quantile=[0.01, 0.99], symmetrical=True)

### Least-squares model for LOS Displacement, mm

In [ ]:
velocity_ps = sbas.velocity(disp_ps)
velocity_ps

In [ ]:
# optionally, materialize to disk and open
velocity_ps = sbas.sync_cube(velocity_ps, 'velocity_ps')

In [ ]:
fig = plt.figure(figsize=(12,4), dpi=300)

zmin, zmax = np.nanquantile(velocity_ps, [0.01, 0.99])
zminmax = max(abs(zmin), zmax)

ax = fig.add_subplot(1, 2, 1)
velocity_ps.plot.imshow(cmap='turbo', vmin=-zminmax, vmax=zminmax, ax=ax)
sbas.geocode(AOI.boundary).plot(ax=ax)
sbas.geocode(POI).plot(ax=ax, marker='x', c='r', markersize=100, label='POI')
ax.set_aspect('auto')
ax.set_title('Velocity, mm/year', fontsize=16)

ax = fig.add_subplot(1, 2, 2)
sbas.as_geo(sbas.ra2ll(velocity_ps)).rio.clip(AOI.geometry)\
    .plot.imshow(cmap='turbo', vmin=-zminmax, vmax=zminmax, ax=ax)
AOI.boundary.plot(ax=ax)
POI.plot(ax=ax, marker='x', c='r', markersize=100, label='POI')
ax.legend(loc='upper left', fontsize=14)
ax.set_title('Velocity, mm/year', fontsize=16)

plt.suptitle('PS LOS Velocity, 2021', fontsize=18)
plt.tight_layout()
plt.show()

### STL model for LOS Displacement, mm

In [ ]:
plt.figure(figsize=(12, 4), dpi=300)

x, y = [(geom.x, geom.y) for geom in sbas.geocode(POI).geometry][0]
disp_pixel = disp_ps.sel(y=y, x=x, method='nearest')
stl_pixel = sbas.stl(disp_ps.sel(y=[y], x=[x], method='nearest')).isel(x=0, y=0)
plt.plot(disp_pixel.date, disp_pixel, c='r', lw=2, label='Displacement POI')
plt.plot(stl_pixel.date, stl_pixel.trend, c='r', ls='--', lw=2, label='Trend POI')
plt.plot(stl_pixel.date, stl_pixel.seasonal, c='r', lw=1, label='Seasonal POI')

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=14)
plt.title('PS LOS Displacement STL Decompose, 2021', fontsize=18)
plt.ylabel('Displacement, mm', fontsize=16)
plt.show()

In [ ]:
x, y = [(geom.x, geom.y) for geom in sbas.geocode(POI).geometry][0]
sbas.plot_baseline_displacement_los_mm(disp_ps_pairs.sel(y=y, x=x, method='nearest')/sbas.los_displacement_mm(1),
                                corr_ps.sel(y=y, x=x, method='nearest'),
                               caption='POI', stl=True)

### RMSE Error Estimation

In [ ]:
rmse_ps = sbas.rmse(disp_ps_pairs, disp_ps, corr_ps)
rmse_ps

In [ ]:
# optionally, materialize to disk and open
rmse_ps = sbas.sync_cube(rmse_ps, 'rmse_ps')

In [ ]:
sbas.plot_rmse(rmse_ps, caption='RMSE Correlation Aware, [mm]')

## SBAS vs PS Comparision

In [ ]:
# crop AOI
points_sbas = sbas.as_geo(sbas.ra2ll(velocity_sbas)).rio.clip(AOI.geometry)
points_ps = sbas.as_geo(sbas.ra2ll(velocity_ps)).rio.clip(AOI.geometry)
points_ps = points_ps.interp_like(points_sbas, method='nearest').values.ravel()
points_sbas = points_sbas.values.ravel()
nanmask = np.isnan(points_sbas) | np.isnan(points_ps)
points_sbas = points_sbas[~nanmask]
points_ps = points_ps[~nanmask]

In [ ]:
plt.figure(figsize=(12, 4), dpi=300)
plt.scatter(points_sbas, points_ps, c='silver', alpha=1,   s=1)
plt.scatter(points_sbas, points_ps, c='b',      alpha=0.1, s=1)
plt.scatter(points_sbas, points_ps, c='g',      alpha=0.1, s=0.1)
plt.scatter(points_sbas, points_ps, c='y',      alpha=0.1, s=0.01)

# adding a 1:1 line
max_value = max(velocity_sbas.max(), velocity_ps.max())
min_value = min(velocity_sbas.min(), velocity_ps.min())
plt.plot([min_value, max_value], [min_value, max_value], 'k--')

plt.xlabel('Velocity SBAS, mm/year', fontsize=16)
plt.ylabel('Velocity PS, mm/year', fontsize=16)
plt.title('Cross-Comparison between SBAS and PS Velocity', fontsize=18)
plt.grid(True)
plt.show()

## 3D Interactive Map

In [ ]:
velocity_sbas_ll = sbas.ra2ll(velocity_sbas)
velocity_ps_ll = sbas.ra2ll(velocity_ps)

velocity_sbas_ll = sbas.as_geo(velocity_sbas_ll).rio.clip(AOI.geometry.envelope)
velocity_ps_ll = sbas.as_geo(velocity_ps_ll).rio.clip(AOI.geometry.envelope)

In [ ]:
gmap = XYZTiles().download(velocity_sbas_ll, 15)

In [ ]:
sbas.export_vtk(velocity_sbas_ll[::3,::2], 'velocity_sbas', image=gmap)
sbas.export_vtk(velocity_ps_ll[::3,::8],   'velocity_ps',   image=gmap)

In [ ]:
plotter = pv.Plotter(shape=(1, 2), notebook=True)
axes = pv.Axes(show_actor=True, actor_scale=2.0, line_width=5)

plotter.subplot(0, 0)
vtk_grid = pv.read('velocity_sbas.vtk')
mesh = vtk_grid.scale([1, 1, 0.00001]).rotate_z(135, point=axes.origin)
plotter.add_mesh(mesh.scale([1, 1, 0.999]), scalars='colors', rgb=True, ambient=0.2)
plotter.add_mesh(mesh, scalars='trend', ambient=0.2, cmap='turbo', clim=(-100,100), nan_opacity=0.1, nan_color='black')
plotter.show_axes()
plotter.add_title('SBAS LOS Velocity', font_size=32)

plotter.subplot(0, 1)
vtk_grid = pv.read('velocity_ps.vtk')
mesh = vtk_grid.scale([1, 1, 0.00001]).rotate_z(135, point=axes.origin)
plotter.add_mesh(mesh.scale([1, 1, 0.999]), scalars='colors', rgb=True, ambient=0.2)
plotter.add_mesh(mesh, scalars='trend', ambient=0.2, cmap='turbo', clim=(-100,100), nan_opacity=0.1, nan_color='black')
plotter.show_axes()
plotter.add_title('PS LOS Velocity', font_size=32)

plotter.show_axes()
plotter._on_first_render_request()
panel.panel(
    plotter.render_window, orientation_widget=plotter.renderer.axes_enabled,
    enable_keybindings=False, sizing_mode='stretch_width', min_height=600
)

## Export VTK file from Google Colab

In [ ]:
if 'google.colab' in sys.modules:
    from google.colab import files
    files.download('velocity_sbas.vtk')
    files.download('velocity_ps.vtk')